Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC3641 - Aprendizaje Automático Basado en Grafos <br>
Segundo Semestre 2025<br>


<h1><center> Tarea 2  </center></h1>
        Profesor: Marcelo Mendoza<br>
        Fecha de entrega: 3 de octubre


---

## Indicaciones

Se debe entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas.

**IMPORTANTE**:
- Se asignará puntaje por el código implementado y los comentarios asociados a resultados.
- El notebook debe tener todas las celdas de código ejecutadas.
- Cualquier instancia de copia resultará en un 1.1 como nota de curso.

---

# Integrantes del grupo

* Estudiante 1: Vittorio Salvatore | 21639795
* Estudiante 2:

# Librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import networkx as nx

# Parte 1: Node2Vec (20 puntos)

En esta sección se debe trabajar con el dataset **PubMed**.

Este conjunto de datos está disponibles en pytorch-geometric. Ver enlace: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.datasets.Planetoid.html#torch_geometric.datasets.Planetoid

A continuación, se presenta el código para cargar el dataset.

In [ ]:
from torch_geometric.datasets import Planetoid

In [ ]:
dataset = Planetoid('./data/PubMed', 'PubMed') #definir root y el dataset que desea descargar
data = dataset[0]

Processing...
Done!


##1.1 Conceptos básicos (3 puntos)

Responda las siguientes preguntas:

1. Explique el concepto de **Random Walk**. ¿Qué rol cumple en la construcción de los embeddings?

2.  Explique el concepto **Biased Random Walk**. ¿En qué se diferencia de **Random Walk**? Explique el impacto que tienen los parámetros **p** y **q** para este método.

Respuesta:

**Random Walk**:

Dado un grafo G = (V, E), direccionado o no, con V conjunto de vertices y E conjunto de aristas, se le llama `random walk` a un cadena de markov que se mueve entre los nodos del grafo a través de las aristas con distribución uniforme entre los posibles caminos disponibles en cada paso (nodo).

A través de los `random walks`, se pueden contruir secuencias de caminos posibles en el grafo, codificando información de su estructura en formato secuencial. De este modo, los `random walks` pueden ser tratados como las oraciones de word2vec, con los nodos siendo el vocabulario y el largo de la walk el tamaño del contexto, y finalmente utilizar esta información para los embeddings.

**Biased Random Walks**:

Una `biased random walk` es un tipo de `random walk` que no distribuye de manera uniforme en cuanto a su siguiente paso. Utiliza información del nodo anterior y del actual y, dependiendo de sus parámetros, **p** y **q**, genera una distribución de probabilidad. Esta tiene el formato:

$$
\alpha_{p,q}\!\left(v^{(t+1)} \;\middle|\; v^{(t)}, v\right) =
\begin{cases}
\frac{1}{p}, & d\!\left(v^{(t-1)}, v^{(t+1)}\right) = 0 \\[6pt]
1, & d\!\left(v^{(t-1)}, v^{(t+1)}\right) = 1 \\[6pt]
\frac{1}{q}, & d\!\left(v^{(t-1)}, v^{(t+1)}\right) = 2
\end{cases}
$$

En esta distribución de probabilidad, el parámetro **`p`** controla la probabilidad de revisitar el nodo $v^{t-1}$ (devolverse), con un **`p`** < 1 sesgando a favor de devolverse y **`p`** > 1 no devolverse. Por otro lado, **`q`** > 1 lleva a sesgar la random walk a visitar nodos cercanos (distancia de 1) y **`q`** < 1 lleva a alejarse del vecindario. Notar que esto significa que las `biased random walks` se comportan más como producto de un algoritmo BFS y DFS con **`q`** > 1 y **`q`** < 1 respectivamente. Finalmente, para que sea una distribución de probabilidad, cabe destacar que se normaliza $\alpha$.


##1.2 Análisis descriptivo (3 puntos)

Grafique la red y calcule medidas descriptivas para caracterizar el grafo.

Comente sus resultados.

**Observación**

Dada la cantidad de nodos, graficar la red puede tomar cerca de 20 minutos.

Respuesta:

##1.3 Crear embeddings (8 puntos)

Utilice el modelo Node2Vec para crear 3 representaciones vectoriales de los nodos de la red, modificando el hiperparámetro: ```walk_length=[10, 30, 50]```. Estas representaciones deben tener una dimensionalidad de 128.

Considere un entrenamiento de 10 épocas

Luego, para cada caso, aplique la técnica de reducción de dimensionalidad **Principal component analysis (PCA)** y refleje el comportamiento de los embeddings creados en gráficos 2D. Distinga cada clase con un color diferente. Comente sus resultados.



**Observación**

Se recomienda trabajar con GPU para reducir el tiempo de cómputo.


Respuesta:

##1.4 Entrenar clasificador (6 puntos)

Considerando las tres representaciones creadas para cada grafo, variando el hiperparámetro ```walk_length```, entrene clasificadores multiclase a nivel de nodos. Para ello, considere un método de machine learning.

Utilice un 80% para train y un 20% para test, manteniendo fija la semilla entre todos los conjuntos creados (```random_state``` fijo).

Calcule las métricas precision, recall y F1-score para cada clase.
Compare los resultados obtenidos.

Luego, evalúe el impacto de modificar el hiperparámetro ```walk_length``` en las métricas de evaluación. ¿Qué impacto tiene?

Respuesta:

#Parte 2: Graph Convolutional Networks (GCN) (20 puntos)

En esta segunda sección se trabajará con dos datasets:

* CiteseerGraphDataset: https://www.dgl.ai/dgl_docs/generated/dgl.data.CiteseerGraphDataset.html#dgl.data.CiteseerGraphDataset

* PubmedGraphDataset: https://www.dgl.ai/dgl_docs/generated/dgl.data.PubmedGraphDataset.html#dgl.data.PubmedGraphDataset

Para instalar dgl en google colab, deben obtener el código desde el siguiente sitio: https://www.dgl.ai/pages/start.html

**Observación**

* Deben utilizar la separación train, val y test que vienen por defecto. Para cada caso, este indicador está reflejado en las variables **train_mask**, **val_mask** y **test_mask** que está disponible al cargar el conjunto de datos.

* Para trabajar con el código visto en clases debe ajustar la versión de pytorch.

Respuesta:

## 2.1 Conceptos básicos (4 puntos)

Responda las siguiente preguntas.

1. Explique los conceptos **AGGREGATE** y **COMBINE** en el contexto de Graph Neural Networks. ¿Qué rol cumplen en la creación de embeddings?


2. ¿Qué relación existe entre **Graph Convolutional Networks** y **Spectral Graph Convolutions**? ¿Qué ventaja tiene GCN?

Respuesta:

## 2.2 Análisis descriptivo (4 puntos)

Grafique los grafos **CiteseerGraphDataset** y **PubmedGraphDataset** y calcule medidas descriptivas.

Compare ambas redes.

Respuesta:

##2.3 GCN CiteseerGraphDataset (6 puntos)

Cree un clasificador a nivel de nodos para el conjunto **CiteseerGraphDataset**, utilizando una red GCN. Defina el número de épocas de manera que se garantice la convergencia del entrenamiento y trabaje con los conjuntos ya definidos de train y val. Grafique la función de pérdida y accuracy. Comente sus resultados.

Luego, obtenga las métricas Precision, Recall y F1-score por clase y a nivel general, considerando los conjuntos de val y test de manera independiente. ¿Qué tan bien generaliza el modelo?

Respuesta:

##2.4 GCN PubmedGraphDataset (6 puntos)

Cree un clasificador a nivel de nodos para el conjunto **PubmedGraphDataset**, utilizando una red GCN. Defina el número de épocas de manera que se garantice la convergencia del entrenamiento y trabaje con los conjuntos ya definidos de train y val. Grafique la función de pérdida y accuracy. Comente sus resultados.

Luego, obtenga las métricas Precision, Recall y F1-score por clase y a nivel general, considerando los conjuntos de val y test de manera independiente. ¿Qué tan bien generaliza el modelo?

Respuesta:

# Parte 3: Graph Attention Networks (GAT) (16 puntos)

En esta sección debe continuar trabajando con los dos conjuntos de datos utilizados en la Parte 2: **CiteseerGraphDataset** y **PubmedGraphDataset**.

## 3.1 Conceptos básicos (4 puntos)

Responda las siguiente preguntas.

1. Explique el concepto de **atención neuronal** en el contexto de grafos. ¿Qué rol cumple el uso de **multi-head attention**?

2. ¿En qué tipo de grafos **GAT** suele funcionar mejor que **GCN**? ¿En qué situaciones sería preferible usar **GCN** en vez de **GAT**?

Respuesta:

## 3.2 GAT CiteseerGraphDataset (6 puntos)

Cree un clasificador a nivel de nodos para el conjunto **CiteseerGraphDataset**, utilizando una red GAT. Defina el número de épocas de manera que se garantice la convergencia del entrenamiento y trabaje con los conjuntos ya definidos de train y val. Grafique la función de pérdida y accuracy. Comente sus resultados.

Luego, obtenga las métricas Precision, Recall y F1-score por clase y a nivel general, considerando los conjuntos de val y test de manera independiente. ¿Qué tan bien generaliza el modelo?



Respuesta:

##3.3 GAT PubmedGraphDataset (6 puntos)

Cree un clasificador a nivel de nodos para el conjunto **PubmedGraphDataset**, utilizando una red GAT. Defina el número de épocas de manera que se garantice la convergencia del entrenamiento y trabaje con los conjuntos ya definidos de train y val. Grafique la función de pérdida y accuracy. Comente sus resultados.

Luego, obtenga las métricas Precision, Recall y F1-score por clase y a nivel general, considerando los conjuntos de val y test de manera independiente. ¿Qué tan bien generaliza el modelo?

Respuesta:

# Parte 4: Conclusiones generales GCN vs GAT (4 puntos)

Compare los resultados obtenidos en las partes 2 y 3. Presente sus conclusiones sobre la experimentación realizada.

Respuesta: